In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

# Fifth Exercise Session

In [2]:
import pandas as  pd
import numpy as np

## Similarity measures

The following similarity functions are used to find duplicates.

### String-based similarity functions

#### Levenstein similarity distances

Compares two words character by character

In [3]:
def LD(s, t):
  if s == "":
    return len(t)
  if t == "":
    return len(s)
  if s[-1] == t[-1]:
    cost = 0
  else:
    cost = 1
  
  res = min([LD(s[:-1], t)+1,
             LD(s, t[:-1])+1,
             LD(s[:-1], t[:-1]) + cost])
  return res

In [4]:
s = "Maria"
t = "Mariella"
distance = LD(s,t)
distance

3

Aligning the words we obtain "Mari---a" for Maria and "Mari---a" for Mariella since the common letters must be aligned. Hence to go from Maria to Mariella you have to do 3 insertion (or 3 deletion from Mariella to Maria).

This is not a similarity but a distance, the number of operation to change one word in the other.

The similarity is defined as

$1 - \frac{LD(s1, s2)}{max(|s1|, |s2|)}$

In [5]:
max_len = max(len(s), len(t))
similarity = 1 - (distance/max_len)
similarity

0.625

We can use a library to implement the Leveinstein distance

In [6]:
!pip install Levenshtein
import Levenshtein as lev

In [7]:
distance = lev.distance(s, t)
distance

3

In [8]:
ratio = lev.ratio(s, t)
ratio

0.7692307692307692

The similarity value is different. Checking on the internet  seems that the formula is the same but it gives different results. The problems may come from some strange computation inside the ratio function.

#### Jaro-winkler

In [9]:
!pip install jaro-winkler
import jaro

In [10]:
jaro.jaro_winkler_metric('Maria', 'Mariella')

0.925

The similarity is high since the common letter are already in the same order, so there is no need of trnasposition to obtain the same order of the common letters.

Even if they have a different length this does not have an impact on the measure, what counts is only te order of the common character.

#### Soundex

Do not use the library called soundex since it has some problems, use instead:

In [11]:
!pip install pyphonetics
import pyphonetics as pyph

In [12]:
soundex = pyph.Soundex()
print(soundex.phonetics('Maria'))
print(soundex.phonetics('Mariella'))

M600
M640


The code is different since it is created picking the first letter of the word and then considering the first 3 consonant to define three numbers according to a fixed mapping from consonant to number.

In [13]:
soundex.sounds_like('Maria', 'Mariella')

False

This function return true if the two words have the same sound i.e. the same soundex code and false otherwise.

In [14]:
print(soundex.sounds_like('London', 'Londonnnn'))
print(soundex.sounds_like('London', 'Londonttt'))

True
True


Is true since the soundex code does not consider doubles and more than 3 consonant.

When we have different soundex code we can compute the similarity comparing the two soundex codes. The sounds_like() function is too strict, we want to know if they are similar, not necessarily equal.

In [15]:
soundex.distance('Maria', 'Mariella', metric = 'levenshtein')

1

The latter returned the distance between the two soundex code (in particular the levenshtein one, but different metrics can be specified).

This is useful for real application where you may want to compare the codes, understanding how close they are.

### Item-based similarity functions

#### Jaccard

$\frac{|A \cap B|}{|A \cup B}|$

In [16]:
a = [0, 1, 2, 5, 6, 8, 9]
b = [0, 2, 3, 4, 5, 7, 9]

In [17]:
def jaccard(list1, list2):
  instersection_len = len(list(set(list1).intersection(list2)))
  union_len = len(list1) + len(list2) - instersection_len
  return float(instersection_len / union_len)

In [18]:
jaccard(a, b)

0.4

The computation of the distance is easy since it consider the list as a bag of words considering the common elements in the two normalized by the whole set of words.

Jaccard distance is useful to compare two documents, seen as two bag of words:

In [19]:
doc1 = {'data', 'are', 'valuable', 'assets', 'of', 'the', 'company'}
doc2 = {'data', 'are', 'assets'}
jaccard(doc1, doc2)

0.42857142857142855

#### TF-IDF

It is used more frequently than Jaccard to compare text.

The words are modeled as vector and then are compared. 

Furthermore, it gives more weight to frequent words inside a text but also decrease the weight of words really common in the union of the documents.

In [20]:
corpus = ['Data are valuable assets of the company',
          'Data are assets',
          'Data quality is important for a company',
          'Data Quality is important for data scientists',
          'Data are valuable products of the company'
         ]

In [21]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

In [22]:
tfidf_vectorizer = TfidfVectorizer()

Then the different vectors must be computed. They are stored into a matrix

In [23]:
tfidf_matrix = tfidf_vectorizer.fit_transform(corpus)

Then the cosine similarity can be used to find the similarity between the different vectors that represent the different texts in the corpus.

In [24]:
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
cosine_sim

array([[1.        , 0.59652436, 0.19322415, 0.11072936, 0.78925825],
       [0.59652436, 1.        , 0.10886738, 0.18562421, 0.29050797],
       [0.19322415, 0.10886738, 1.        , 0.79495102, 0.18476652],
       [0.11072936, 0.18562421, 0.79495102, 1.        , 0.10588262],
       [0.78925825, 0.29050797, 0.18476652, 0.10588262, 1.        ]])

Each row and column correspond to a text of the corpus: it concide with the index inside the corpus vector (indeed on the diagonal there are only 1 since each element is equal to itself).

## Deduplication Detection

In [51]:
data = pd.read_csv('./Dataset/iris.csv')

We already have seen how to find the **exact** tuple duplicates of the dataset.

In [52]:
data.duplicated()

0      False
1      False
2      False
3      False
4      False
       ...  
145    False
146    False
147    False
148    False
149    False
Length: 150, dtype: bool

In [53]:
print(data.duplicated().any())
print(data.duplicated().sum())

True
1


In [54]:
data[data.duplicated()]

,sepal.length,sepal.width,petal.length,petal.width,variety
142,5.8,2.7,5.1,1.9,Virginica


If we want to drop it we can do

In [55]:
data = data.drop_duplicates()

In [56]:
data.duplicated().any()

False

Most of the times duplicates are not **exactly** matching but they are **similar**. Hence we have to use techniques such as data deduplication (single source) or recorlinkage/object identification (two sources). 

### Data Deduplication

This techniques involve data deduplication accross two sources.

In [57]:
!pip install recordlinkage
import recordlinkage
from recordlinkage.datasets import load_febrl1 # there are other dataset that have an increased volume

In [58]:
data1 = load_febrl1()
data1.head(10)

,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id
rec_id,,,,,,,,,,
rec-223-org,NaN,waller,6,tullaroop street,willaroo,st james,4011,wa,19081209,6988048
rec-122-org,lachlan,berry,69,giblin street,killarney,bittern,4814,qld,19990219,7364009
rec-373-org,deakin,sondergeld,48,goldfinch circuit,kooltuo,canterbury,2776,vic,19600210,2635962
rec-10-dup-0,kayla,harrington,NaN,maltby circuit,coaling,coolaroo,3465,nsw,19150612,9004242
rec-227-org,luke,purdon,23,ramsay place,mirani,garbutt,2260,vic,19831024,8099933
rec-6-dup-0,NaN,trevorrow,16,dumas street,2/98-latchford barracks,mount immaney,2281,wa,19530313,4811642
rec-190-dup-0,darcie,turtur,10,blacket street,eureka,beverly hills,2263,nsw,NaN,2025650
rec-294-org,william,bishop,21,neworra place,apmnt 65,worongary,6225,qld,19490130,9773843
rec-206-dup-0,NaN,lombardi,36,yerralee road,leisure living vlge,carlsruhe,3149,qld,19870919,1613132


In [61]:
data1.shape

(1000, 10)

In [62]:
data1.duplicated().any()

False

There are not **exact** duplicates.

The first thing we have to do is to build the set of tuple pairs to compare (in the package is called indexing)

In [63]:
indexer = recordlinkage.Index()
indexer.full() # considering all the possible pairs (=> WARNING A full index can result in large number of record pairs.)
candidate_links = indexer.index(data1)

<Index>

The possible record pairs are: $\frac{1000^2 - 1000}{2} = 499500$ since we consider all the combinations ($1000^2$) but not the ones between the same record ($-1000$), considering that the comparison between two record should not be considered two times since it is symmetric ($/2$).

In [64]:
print(len(candidate_links))

499500


How we can minimize the number of comparison i.e. to minimize the number of candidate links?

*   Blocking
*   Sorted Neighborhood (sliding window)



#### Comparison set: Blocking

Do not consider the record that surely do not match, and this can be done considering blocks of tuples that agree on the value of an attribute.

Let's consider block considering the first name 

In [121]:
indexer = recordlinkage.Index()
indexer.block('given_name')
candidate_links = indexer.index(data1)

<Index>

In [122]:
print(len(candidate_links))

2082


In [123]:
indexer = recordlinkage.Index()
indexer.block('surname')
candidate_links = indexer.index(data1)

<Index>

In [124]:
print(len(candidate_links))

1707


Considering the blocking using the surname we obtain less candidate link since the surname is discriminating more than the name.

In [125]:
indexer = recordlinkage.Index()
indexer.block('given_name', 'surname')
candidate_links = indexer.index(data1)

<Index>

In [126]:
print(len(candidate_links))

197


Considering the blocking using both name and  surname we obtain even less candidate links.

The **disdvantage** is that is not possible to compare tuples in twoo different blocks, so there can be problems in the identification of all the duplicates.

In [127]:
#indexer = recordlinkage.BlockIndex(on='given_name')

#### Comparison set: Sorted Neighborhood

In [128]:
indexer = recordlinkage.SortedNeighbourhoodIndex('given_name', window = 9)
candidate_links = indexer.index(data1)

In [129]:
print(len(candidate_links))

11447


Increasing the window length we expect the candidate link to increase since you can do more comparisons.

Instead decreasing the window length we expect the candidate link to decrease since you can do more comparisons.

In [130]:
indexer = recordlinkage.SortedNeighbourhoodIndex('given_name', window = 15)
candidate_links = indexer.index(data1)

In [131]:
print(len(candidate_links))

19036


Usually to define the length of the window we look for the number of comparison that we want to do and try to match them.

### Comparison

In [132]:
compare_cl = recordlinkage.Compare()

In [133]:
compare_cl.exact('given_name', 'given_name', label = 'given_name') # repeat the attribute since the function is able to work with 2 different datasets

<Compare>

There are function to evaluate similarity of different types of data (e.g. strings, dates,...)

In [134]:
compare_cl.string('surname', 'surname', method='jarowinkler', threshold=0.85, label='surname')

<Compare>

In [135]:
compare_cl.exact('date_of_birth', 'date_of_birth', label = 'date_of_birth')

<Compare>

In [136]:
compare_cl.exact('suburb', 'suburb', label = 'suburb')

<Compare>

In [137]:
compare_cl.exact('state', 'state', label = 'state')

<Compare>

In [138]:
compare_cl.string('address_1', 'address_1', threshold=0.85, label='address_1')

<Compare>

In [139]:
features = compare_cl.compute(candidate_links, data1)
features.head(20)

## ATTENTION: the result is a little different from the one otained by the prof. 
## Everything was copied correctly 

given_name  surname  date_of_birth  suburb  \
rec_id_1      rec_id_2                                                    
rec-469-dup-0 rec-7-org               0      0.0              0       0   
rec-373-dup-0 rec-94-org              0      0.0              0       0   
              rec-94-dup-0            0      0.0              0       0   
rec-10-org    rec-285-org             0      0.0              0       0   
rec-342-org   rec-285-org             0      0.0              0       0   
rec-342-dup-0 rec-285-org             0      0.0              0       0   
rec-472-org   rec-321-dup-0           0      0.0              0       0   
rec-330-org   rec-321-dup-0           0      0.0              0       0   
rec-474-org   rec-386-dup-0           0      0.0              0       0   
              rec-386-org             0      0.0              0       0   
rec-26-org    rec-335-org             0      0.0              0       0   
              rec-335-dup-0           0      0.0              0       0   
              rec-316-dup-0           0      0.0              0       0   
              rec-316-org             0      0.0              0       0   
rec-81-org    rec-335-org             0      0.0              0       0   
              rec-335-dup-0           0      0.0              0       0   
              rec-316-dup-0           0      0.0              0       0   
              rec-316-org             0      0.0              0       0   
rec-385-dup-0 rec-67-org              0      0.0              0       0   
              rec-284-dup-0           0      0.0              0       0   

                             state  address_1  
rec_id_1      rec_id_2                         
rec-469-dup-0 rec-7-org          0        0.0  
rec-373-dup-0 rec-94-org         0        0.0  
              rec-94-dup-0       0        0.0  
rec-10-org    rec-285-org        0        0.0  
rec-342-org   rec-285-org        0        0.0  
rec-342-dup-0 rec-285-org        0        0.0  
rec-472-org   rec-321-dup-0      0        0.0  
rec-330-org   rec-321-dup-0      0        0.0  
rec-474-org   rec-386-dup-0      1        0.0  
              rec-386-org        1        0.0  
rec-26-org    rec-335-org        0        0.0  
              rec-335-dup-0      0        0.0  
              rec-316-dup-0      0        0.0  
              rec-316-org        0        0.0  
rec-81-org    rec-335-org        1        0.0  
              rec-335-dup-0      1        0.0  
              rec-316-dup-0      0        0.0  
              rec-316-org        0        0.0  
rec-385-dup-0 rec-67-org         0        0.0  
              rec-284-dup-0      0        0.0

The zeros and the one tell us if the tuples satisfy the comparison function or not. 

In [140]:
features.describe()

,given_name,surname,date_of_birth,suburb,state,address_1
count,19036.000000,19036.000000,19036.000000,19036.000000,19036.000000,19036.000000
mean,0.109372,0.021486,0.017756,0.014604,0.248056,0.017493
std,0.312113,0.145000,0.132066,0.119964,0.431896,0.131103
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


With the latter function we could see, looking at the count row, where the matching of the comperison are.

We can count the sum of boolean values to know if there are some matches or not:

In [141]:
features.sum(axis=1).value_counts().sort_index(ascending=False)

6.0      142
5.0      167
4.0       55
3.0       15
2.0      396
1.0     5418
0.0    12843
dtype: int64

Hence in the dataset we have 142 comparisons that has a row, in the feature matrix, full of 1, so they are clearly duplicates

In [143]:
matches = features[features.sum(axis=1) > 4]
len(matches)

309

In [144]:
matches.head()

,,given_name,surname,date_of_birth,suburb,state,address_1
rec_id_1,rec_id_2,,,,,,
rec-285-dup-0,rec-285-org,0,1.0,1,1,1,1.0
rec-491-dup-0,rec-491-org,0,1.0,1,1,1,1.0
rec-439-dup-0,rec-439-org,0,1.0,1,1,1,1.0
rec-71-org,rec-71-dup-0,0,1.0,1,1,1,1.0
rec-423-dup-0,rec-423-org,0,1.0,1,1,1,1.0


Which is the list of recors that are likely to be duplicates.

Hence this is a method to detect if the tuples are duplicated through the similarity measures. Is possible to define a similarity measure on all the attributes.